In [7]:
from langchain.llms import GooglePalm

api_key="AIzaSyBAOvmnwsVft3mAPpgbrxNu"

llm=GooglePalm(google_api_key=api_key,temperature=0.6)

In [8]:
import openai
import pandas as pd

OPENAI_API_KEY="sk-skCNUTE0sYTTeMJK5zpCT3BlbkFJkh96lIm"
openai.api_key=OPENAI_API_KEY

In [3]:
from langchain.document_loaders.csv_loader import CSVLoader

loader=CSVLoader(file_path='common_faqs.csv',source_column='prompt',encoding='ISO-8859-1')
data=loader.load()


In [13]:
from langchain.embeddings import OpenAIEmbeddings

instructor_embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [37]:
from langchain.vectorstores import FAISS

vector_db=FAISS.from_documents(documents=data,embedding=instructor_embeddings)


In [38]:
retriever=vector_db.as_retriever()
rdocs=retriever.get_relevant_documents("is there any job assitant program")
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to p

In [28]:
from langchain.prompts import PromptTemplate

prompt_template="""Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT=PromptTemplate(
    template=prompt_template,input_variables=['context','question']
)

In [29]:
from langchain.chains import RetrievalQA

chain=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",
                        retriever=retriever,
                        input_key="query",
                        return_source_documents=True,
                        chain_type_kwargs={'prompt':PROMPT})

In [36]:
chain("powerbi or tableu and is there any internships?")

{'query': 'powerbi or tableu and is there any internships?',
 'result': 'This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation features are available in both. However, factually speaking Power BI is cheaper and offers tighter integration with the Microsoft environment. Since most companies use excel & Microsoft tools they start with Power BI or move towards Power BI for seamless integration with other Microsoft tools (called as Power platform). This makes the job openings grow at a much higher rate on Power BI and Power Platform. Also, Power BI has been leading the Gartner’s magic quadrant in BI for the last few years as the industry leader.\n\nYes, we do provide virtual internships.',
 'source_documents': [Document(page_content='prompt: Power BI or Tableau which one is better?\nresponse: This is a contextual question. If you are talking about a pure visualization tool Tableau is slight